In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
import xgboost as xgb
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings("ignore")

In [7]:
# ETH Price Prediction - Data Analysis & Model Development
# This notebook contains the complete analysis pipeline

# %% [markdown]
# # Ethereum Price Prediction Using Whale Transaction Analysis
# 
# ## Objective
# Predict Ethereum price movements based on:
# - Whale transaction patterns
# - Exchange flows
# - Market indicators
# - Bitcoin correlation metrics

# %% [markdown]
# ## 1. Import Libraries

# %%
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import requests
from dotenv import load_dotenv
from dune_client.client import DuneClient

warnings.filterwarnings('ignore')

# Sklearn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report, confusion_matrix
)

# ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb

# Plotting settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("Libraries imported successfully!")



Libraries imported successfully!


In [11]:

load_dotenv()
dune = DuneClient("$DUNE_WHALES_MOVES")
API_KEY = os.getenv("COINGECKO_API_KEY")


In [12]:
import time
from dotenv import load_dotenv

# --- Load environment variables ---
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_WHALES_API")

if not DUNE_API_KEY:
    raise ValueError("❌ Missing DUNE_API_KEY. Make sure it's in your .env file.")

# --- Query setup ---
QUERY_ID = "6184996"
headers = {"x-dune-api-key": DUNE_API_KEY}

# --- 1. Trigger new execution ---
execute_url = f"https://api.dune.com/api/v1/query/{QUERY_ID}/execute"
execute_response = requests.post(execute_url, headers=headers)
execute_data = execute_response.json()
print("🔍 Raw response from Dune:", execute_data)

execution_id = execute_data.get("execution_id")
if not execution_id:
    raise ValueError(f"❌ No execution_id found. Response:\n{execute_data}")

print(f"🚀 Query started — Execution ID: {execution_id}")

# --- 2. Poll for completion ---
status_url = f"https://api.dune.com/api/v1/execution/{execution_id}/status"
results_url = f"https://api.dune.com/api/v1/execution/{execution_id}/results"

while True:
    status_response = requests.get(status_url, headers=headers)
    state = status_response.json().get("state")
    print("State:", state)

    if state == "QUERY_STATE_COMPLETED":
        print("✅ Query complete!")
        break
    elif state == "QUERY_STATE_FAILED":
        raise RuntimeError("❌ Query failed:", status_response.json())
    else:
        time.sleep(10)

# --- 3. Fetch results ---
results_response = requests.get(results_url, headers=headers)
results_data = results_response.json()

rows = results_data["result"]["rows"]
df_whales = pd.DataFrame(rows)
print(f"✅ Retrieved {len(df_whales)} rows")
print(df_whales.head())


🔍 Raw response from Dune: {'execution_id': '01KA18HJ5VREN3KQ9QJKQT5WEW', 'state': 'QUERY_STATE_PENDING'}
🚀 Query started — Execution ID: 01KA18HJ5VREN3KQ9QJKQT5WEW
State: QUERY_STATE_EXECUTING
State: QUERY_STATE_EXECUTING
State: QUERY_STATE_EXECUTING
State: QUERY_STATE_EXECUTING
State: QUERY_STATE_EXECUTING
State: QUERY_STATE_EXECUTING
State: QUERY_STATE_EXECUTING
State: QUERY_STATE_EXECUTING
State: QUERY_STATE_EXECUTING
State: QUERY_STATE_COMPLETED
✅ Query complete!
✅ Retrieved 1096 rows
   accumulation_phase activity_level  avg_whale_tx_size_weth  bearish_signal  \
0                   0           high              893.224881               0   
1                   0           high              680.238175               1   
2                   0           high              543.278983               0   
3                   0           high              391.003046               0   
4                   0           high              434.838114               0   

                    block

In [17]:
"""
Fetch ETH and BTC prices from CoinGecko PRO and merge with whale data
"""
import requests
from datetime import datetime, timedelta
import time
from typing import Dict, List

# ==========================================
# CONFIGURATION
# ==========================================


COINGECKO_API_KEY = API_KEY  

# CoinGecko Pro API
COINGECKO_API_BASE = 'https://pro-api.coingecko.com/api/v3/coins/{symbol}/market_chart'

# Output file
OUTPUT_FILE = 'whale_prices_ml_ready.csv'

# Rate limiting (Pro tier is more generous, but still be respectful)
REQUEST_DELAY = 0.5  # seconds between requests

print("✓ Configuration loaded")
print(f"  API: {COINGECKO_API_BASE}")
print(f"  Output file: {OUTPUT_FILE}")

✓ Configuration loaded
  API: https://pro-api.coingecko.com/api/v3/coins/{symbol}/market_chart
  Output file: whale_prices_ml_ready.csv


In [18]:
def fetch_coingecko_price_range(coin_id: str, from_date: str, to_date: str) -> pd.DataFrame:
    """
    Fetch price data for specific date range using CoinGecko Pro API
    
    Args:
        coin_id: e.g. 'ethereum' or 'bitcoin'
        from_date: start date 'YYYY-MM-DD'
        to_date: end date 'YYYY-MM-DD'
    
    Returns:
        DataFrame with daily prices
    """
    print(f"\nFetching {coin_id.upper()} price data from {from_date} to {to_date}...")
    
    # Convert dates to Unix timestamps
    from_ts = int(pd.Timestamp(from_date).timestamp())
    to_ts = int(pd.Timestamp(to_date).timestamp())
    
    # ✅ Use coin_id instead of undefined symbol
    url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    
    headers = {
        'accept': 'application/json',
        'x-cg-pro-api-key': COINGECKO_API_KEY
    }
    
    params = {
        'vs_currency': 'usd',
        'from': from_ts,
        'to': to_ts
    }
    
    print(f"  Request URL: {url}")
    print(f"  Using API key: {COINGECKO_API_KEY[:8]}...{COINGECKO_API_KEY[-4:]}")
    
    try:
        response = requests.get(url, headers=headers, params=params)
        print(f"  Response status: {response.status_code}")
        response.raise_for_status()
        
        data = response.json()
        
        if 'prices' not in data:
            print(f"✗ Unexpected response format: {list(data.keys())}")
            return pd.DataFrame()
        
        prices = data['prices']
        df = pd.DataFrame({
            'timestamp': [p[0] for p in prices],
            'price': [p[1] for p in prices]
        })
        
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
        df = df.groupby('date', as_index=False).agg({'price': 'last'})
        
        print(f"✓ Fetched {len(df)} days of {coin_id.upper()} data")
        print(f"  Price range: ${df['price'].min():.2f} - ${df['price'].max():.2f}")
        return df
        
    except requests.exceptions.HTTPError as e:
        print(f"✗ HTTP Error fetching {coin_id} data: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"  Response: {e.response.text[:300]}")
        return pd.DataFrame()
    except Exception as e:
        print(f"✗ Error fetching {coin_id} data: {e}")
        return pd.DataFrame()

print("✓ Fixed price fetching function loaded")


✓ Fixed price fetching function loaded


In [19]:
def fetch_coingecko_price_range(coin_id: str, from_date: str, to_date: str) -> pd.DataFrame:
    """
    Fetch price data for specific date range using CoinGecko Pro API
    
    Args:
        coin_id: e.g. 'ethereum' or 'bitcoin'
        from_date: start date 'YYYY-MM-DD'
        to_date: end date 'YYYY-MM-DD'
    
    Returns:
        DataFrame with daily prices
    """
    print(f"\nFetching {coin_id.upper()} price data from {from_date} to {to_date}...")
    
    # Convert dates to Unix timestamps
    from_ts = int(pd.Timestamp(from_date).timestamp())
    to_ts = int(pd.Timestamp(to_date).timestamp())
    
    # ✅ Use coin_id instead of undefined symbol
    url = f"https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
    
    headers = {
        'accept': 'application/json',
        'x-cg-pro-api-key': COINGECKO_API_KEY
    }
    
    params = {
        'vs_currency': 'usd',
        'from': from_ts,
        'to': to_ts
    }
    
    print(f"  Request URL: {url}")
    print(f"  Using API key: {COINGECKO_API_KEY[:8]}...{COINGECKO_API_KEY[-4:]}")
    
    try:
        response = requests.get(url, headers=headers, params=params)
        print(f"  Response status: {response.status_code}")
        response.raise_for_status()
        
        data = response.json()
        
        if 'prices' not in data:
            print(f"✗ Unexpected response format: {list(data.keys())}")
            return pd.DataFrame()
        
        prices = data['prices']
        df = pd.DataFrame({
            'timestamp': [p[0] for p in prices],
            'price': [p[1] for p in prices]
        })
        
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
        df = df.groupby('date', as_index=False).agg({'price': 'last'})
        
        print(f"✓ Fetched {len(df)} days of {coin_id.upper()} data")
        print(f"  Price range: ${df['price'].min():.2f} - ${df['price'].max():.2f}")
        return df
        
    except requests.exceptions.HTTPError as e:
        print(f"✗ HTTP Error fetching {coin_id} data: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"  Response: {e.response.text[:300]}")
        return pd.DataFrame()
    except Exception as e:
        print(f"✗ Error fetching {coin_id} data: {e}")
        return pd.DataFrame()

print("✓ Fixed price fetching function loaded")


✓ Fixed price fetching function loaded


In [20]:
def add_price_features(df: pd.DataFrame, price_col: str, prefix: str) -> pd.DataFrame:
    """
    Add price-based features for ML
    """
    print(f"\nCreating {prefix.upper()} price features...")
    
    # Sort by date to ensure proper ordering
    df = df.sort_values('block_date').reset_index(drop=True)
    
    # Daily returns
    df[f'{prefix}_daily_return'] = df[price_col].pct_change()
    
    # Log returns
    df[f'{prefix}_log_return'] = np.log(df[price_col] / df[price_col].shift(1))
    
    # Price moving averages
    df[f'{prefix}_price_ma7'] = df[price_col].rolling(7, min_periods=1).mean()
    df[f'{prefix}_price_ma30'] = df[price_col].rolling(30, min_periods=1).mean()
    df[f'{prefix}_price_ma90'] = df[price_col].rolling(90, min_periods=1).mean()
    
    # Price vs MA ratios
    df[f'{prefix}_price_vs_ma7'] = df[price_col] / df[f'{prefix}_price_ma7']
    df[f'{prefix}_price_vs_ma30'] = df[price_col] / df[f'{prefix}_price_ma30']
    
    # Volatility
    df[f'{prefix}_volatility_7d'] = df[f'{prefix}_daily_return'].rolling(7, min_periods=1).std()
    df[f'{prefix}_volatility_30d'] = df[f'{prefix}_daily_return'].rolling(30, min_periods=1).std()
    
    # Price range
    df[f'{prefix}_range_7d'] = (
        df[price_col].rolling(7, min_periods=1).max() - 
        df[price_col].rolling(7, min_periods=1).min()
    ) / df[price_col]
    
    # Cumulative returns
    df[f'{prefix}_return_7d'] = df[price_col].pct_change(7)
    df[f'{prefix}_return_30d'] = df[price_col].pct_change(30)
    
    # RSI-like momentum
    returns = df[f'{prefix}_daily_return']
    gains = returns.where(returns > 0, 0).rolling(14, min_periods=1).mean()
    losses = -returns.where(returns < 0, 0).rolling(14, min_periods=1).mean()
    rs = gains / (losses + 1e-10)
    df[f'{prefix}_momentum_rsi'] = 100 - (100 / (1 + rs))
    
    # Trend indicators
    df[f'{prefix}_uptrend'] = (
        (df[price_col] > df[f'{prefix}_price_ma7']) & 
        (df[f'{prefix}_price_ma7'] > df[f'{prefix}_price_ma30'])
    ).astype(int)
    
    df[f'{prefix}_downtrend'] = (
        (df[price_col] < df[f'{prefix}_price_ma7']) & 
        (df[f'{prefix}_price_ma7'] < df[f'{prefix}_price_ma30'])
    ).astype(int)
    
    # Lagged features
    for lag in [1, 3, 7]:
        df[f'{prefix}_return_lag{lag}'] = df[f'{prefix}_daily_return'].shift(lag)
    
    print(f"✓ Created {len([c for c in df.columns if c.startswith(prefix)]) - len([c for c in df.columns if c.startswith(prefix) and 'lag' not in c and 'return' not in c and 'price' not in c])} {prefix.upper()} features")
    
    return df

print("✓ Price feature functions loaded")

✓ Price feature functions loaded


In [21]:
def add_price_features(df: pd.DataFrame, price_col: str, prefix: str) -> pd.DataFrame:
    """
    Add price-based features for ML
    """
    print(f"\nCreating {prefix.upper()} price features...")
    
    # Sort by date to ensure proper ordering
    df = df.sort_values('block_date').reset_index(drop=True)
    
    # Daily returns
    df[f'{prefix}_daily_return'] = df[price_col].pct_change()
    
    # Log returns
    df[f'{prefix}_log_return'] = np.log(df[price_col] / df[price_col].shift(1))
    
    # Price moving averages
    df[f'{prefix}_price_ma7'] = df[price_col].rolling(7, min_periods=1).mean()
    df[f'{prefix}_price_ma30'] = df[price_col].rolling(30, min_periods=1).mean()
    df[f'{prefix}_price_ma90'] = df[price_col].rolling(90, min_periods=1).mean()
    
    # Price vs MA ratios
    df[f'{prefix}_price_vs_ma7'] = df[price_col] / df[f'{prefix}_price_ma7']
    df[f'{prefix}_price_vs_ma30'] = df[price_col] / df[f'{prefix}_price_ma30']
    
    # Volatility
    df[f'{prefix}_volatility_7d'] = df[f'{prefix}_daily_return'].rolling(7, min_periods=1).std()
    df[f'{prefix}_volatility_30d'] = df[f'{prefix}_daily_return'].rolling(30, min_periods=1).std()
    
    # Price range
    df[f'{prefix}_range_7d'] = (
        df[price_col].rolling(7, min_periods=1).max() - 
        df[price_col].rolling(7, min_periods=1).min()
    ) / df[price_col]
    
    # Cumulative returns
    df[f'{prefix}_return_7d'] = df[price_col].pct_change(7)
    df[f'{prefix}_return_30d'] = df[price_col].pct_change(30)
    
    # RSI-like momentum
    returns = df[f'{prefix}_daily_return']
    gains = returns.where(returns > 0, 0).rolling(14, min_periods=1).mean()
    losses = -returns.where(returns < 0, 0).rolling(14, min_periods=1).mean()
    rs = gains / (losses + 1e-10)
    df[f'{prefix}_momentum_rsi'] = 100 - (100 / (1 + rs))
    
    # Trend indicators
    df[f'{prefix}_uptrend'] = (
        (df[price_col] > df[f'{prefix}_price_ma7']) & 
        (df[f'{prefix}_price_ma7'] > df[f'{prefix}_price_ma30'])
    ).astype(int)
    
    df[f'{prefix}_downtrend'] = (
        (df[price_col] < df[f'{prefix}_price_ma7']) & 
        (df[f'{prefix}_price_ma7'] < df[f'{prefix}_price_ma30'])
    ).astype(int)
    
    # Lagged features
    for lag in [1, 3, 7]:
        df[f'{prefix}_return_lag{lag}'] = df[f'{prefix}_daily_return'].shift(lag)
    
    print(f"✓ Created {len([c for c in df.columns if c.startswith(prefix)]) - len([c for c in df.columns if c.startswith(prefix) and 'lag' not in c and 'return' not in c and 'price' not in c])} {prefix.upper()} features")
    
    return df

print("✓ Price feature functions loaded")

✓ Price feature functions loaded


In [15]:
# %%
import pandas as pd

# Show all columns in output
pd.set_option('display.max_columns', None)

# Dataset shape and first rows
print(f"Dataset shape: {df_whales.shape}")
print("\nFirst few rows:")
display(df_whales.head())

# %%
# Data info and missing values
print("Data types and missing values:")
df_whales.info()

# %%
# Basic statistics
print("\nBasic statistics:")
display(df_whales.describe())

# %%
# Target variable distribution
target_col = 'next_day_price_direction'
print(f"\nTarget Variable: {target_col}")
print(df_whales[target_col].value_counts())
print(f"\nClass balance:\n{df_whales[target_col].value_counts(normalize=True)}")

# %%
# Date range
df_whales['block_date'] = pd.to_datetime(df_whales['block_date'])
print(f"\nDate range: {df_whales['block_date'].min()} to {df_whales['block_date'].max()}")
print(f"Number of days: {(df_whales['block_date'].max() - df_whales['block_date'].min()).days}")


Dataset shape: (1096, 87)

First few rows:


,accumulation_phase,activity_level,avg_whale_tx_size_weth,bearish_signal,block_date,bullish_signal,cv_whale_tx_size,data_quality_score,day_of_month,day_of_week,deposit_tx_count,deposit_withdrawal_ratio,distribution_phase,exchange_tx_ratio,extreme_deposit_flag,extreme_withdrawal_flag,high_concentration_flag,inter_exchange_transfers_weth,inter_exchange_tx_count,iqr_whale_tx_size,is_month_end,is_month_start,is_quarter_end,is_weekend,known_whale_ratio,known_whale_tx_count,known_whale_volume_weth,max_whale_tx_size_weth,median_whale_tx_size_weth,mega_whale_dominance_flag,mega_whale_ratio,mega_whale_tx_count,mega_whale_volume_weth,min_whale_tx_size_weth,month,net_flow_direction_change,net_flow_ma30,net_flow_ma7,net_flow_max7,net_flow_min7,net_flow_range_7d,net_flow_vs_ma30,net_flow_vs_ma7,prev_day_net_flow,prev_day_senders,prev_day_tx_count,prev_day_volume,pure_whale_tx_count,pure_whale_volume_weth,q25_whale_tx_size_weth,q75_whale_tx_size_weth,quarter,senders_change_pct,senders_ma7,std_whale_tx_size_weth,sustained_inflow,sustained_outflow,total_whale_volume_weth,tx_count_change_pct,tx_count_ma30,tx_count_ma7,tx_count_vs_ma30,tx_count_vs_ma7,ultra_whale_tx_count,ultra_whale_volume_weth,unique_whale_receivers,unique_whale_senders,unusual_activity_flag,volume_change_pct,volume_downtrend,volume_ma30,volume_ma7,volume_range_pct_7d,volume_spike_30d,volume_spike_7d,volume_std30,volume_std7,volume_uptrend,volume_vs_ma30,volume_vs_ma7,volume_zscore_30d,volume_zscore_7d,whale_exchange_deposits_weth,whale_exchange_withdrawals_weth,whale_net_exchange_flow_weth,whale_tx_count,withdrawal_tx_count
0,0,high,893.224881,0,2025-11-14 00:00:00.000 UTC,0,4.0157,1,14,5,10,3.0684,1,0.0012,1,0,1,0.0,0,0.840674,0,0,0,0,0.8326,8131,10510.6361,24037.8434,0.340103,1,0.0762,744,1.706185e+07,0,11,-1,-5.4666,-33.2004,12.5759,-110.0539,122.6299,5.8139,0.9573,12.5759,1814.0,21156.0,3.188480e+07,9754,1.740532e+07,0.073574,0.914248,4,-0.5943,1176.29,3586.935261,0,0,1.740538e+07,-0.5384,21073.00,18801.00,0.4634,0.5194,510,1.638967e+07,556,736,0,-0.4541,0,1.618050e+07,1.988763e+07,0.9082,0,0,4.884148e+06,6.359017e+06,0,1.0757,0.8752,0.2508,-0.3904,47.1480,15.3657,-31.7823,9766,2
1,0,high,680.238175,1,2025-11-13 00:00:00.000 UTC,0,4.5247,1,13,4,14,0.8571,0,0.0009,0,0,1,0.0,0,0.558779,0,0,0,0,0.8476,17931,17133.1768,110100.7979,0.156293,1,0.0658,1392,3.119731e+07,0,11,1,3.0496,-63.5889,12.5759,-244.5023,257.0783,4.1238,-0.1978,-3.7834,1154.0,18228.0,2.019422e+07,21137,3.188464e+07,0.034242,0.593021,4,0.5719,1213.14,3077.873294,0,0,3.188480e+07,0.1606,21398.80,19415.57,0.9887,1.0896,982,2.974523e+07,969,1814,0,0.5789,0,1.637370e+07,1.818486e+07,1.4517,0,0,5.046215e+06,8.402114e+06,1,1.9473,1.7534,3.0738,1.6305,75.4240,87.9999,12.5759,21156,5
2,0,high,543.278983,0,2025-11-12 00:00:00.000 UTC,0,4.5054,1,12,3,14,1.0211,1,0.0012,0,0,1,0.0,0,0.558595,0,0,0,0,0.8706,15870,15379.2742,28262.0000,0.153166,1,0.0602,1098,1.971304e+07,0,11,0,-10.8346,116.6446,1274.2107,-244.5023,1518.7131,0.3492,-0.0324,-110.0539,1234.0,18518.0,1.467513e+07,18207,2.019386e+07,0.028983,0.587578,4,-0.0648,1093.86,2447.687547,0,1,2.019422e+07,-0.0157,21454.97,18549.43,0.8496,0.9827,775,1.809482e+07,969,1154,0,0.3761,0,1.610240e+07,1.516979e+07,1.2414,0,0,4.354974e+06,6.152122e+06,0,1.2541,1.3312,0.9396,0.8167,182.8842,179.1008,-3.7834,18228,7
3,0,high,391.003046,0,2025-11-11 00:00:00.000 UTC,0,4.4502,1,11,2,25,1.9127,1,0.0021,0,0,1,0.0,0,0.584777,0,0,0,0,0.8686,16085,16359.9813,59999.0000,0.162175,1,0.0586,1085,1.413178e+07,0,11,0,-10.5712,70.7251,1274.2107,-325.2199,1599.4306,10.4107,-1.5561,-11.9611,1158.0,19397.0,1.691303e+07,18480,1.467478e+07,0.031224,0.616001,4,0.0656,1085.14,1740.034756,0,1,1.467513e+07,-0.0453,21672.10,17886.43,0.8545,1.0353,755,1.297065e+07,949,1234,0,-0.1323,0,1.613884e+07,1.378508e+07,1.3661,0,0,4.394785e+06,5.919180e+06,0,0.9093,1.0646,-0.3331,0.1504,230.6338,120.5799,-110.0539,18518,13
4,0,high,434.838114,0,2025-11-10 00:00:00.000 UTC,0,4.5667,1,10,1,16,2.7461,1,0.0011,1,0,1

Data types and missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 87 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   accumulation_phase               1096 non-null   int64  
 1   activity_level                   1096 non-null   object 
 2   avg_whale_tx_size_weth           1096 non-null   float64
 3   bearish_signal                   1096 non-null   int64  
 4   block_date                       1096 non-null   object 
 5   bullish_signal                   1096 non-null   int64  
 6   cv_whale_tx_size                 1096 non-null   float64
 7   data_quality_score               1096 non-null   object 
 8   day_of_month                     1096 non-null   int64  
 9   day_of_week                      1096 non-null   int64  
 10  deposit_tx_count                 1096 non-null   int64  
 11  deposit_withdrawal_ratio         1096 non-null   fl

,accumulation_phase,avg_whale_tx_size_weth,bearish_signal,bullish_signal,cv_whale_tx_size,day_of_month,day_of_week,deposit_tx_count,deposit_withdrawal_ratio,distribution_phase,exchange_tx_ratio,extreme_deposit_flag,extreme_withdrawal_flag,high_concentration_flag,inter_exchange_transfers_weth,inter_exchange_tx_count,iqr_whale_tx_size,is_month_end,is_month_start,is_quarter_end,is_weekend,known_whale_ratio,known_whale_tx_count,known_whale_volume_weth,max_whale_tx_size_weth,median_whale_tx_size_weth,mega_whale_dominance_flag,mega_whale_ratio,mega_whale_tx_count,mega_whale_volume_weth,min_whale_tx_size_weth,month,net_flow_direction_change,net_flow_ma30,net_flow_ma7,net_flow_max7,net_flow_min7,net_flow_range_7d,net_flow_vs_ma30,net_flow_vs_ma7,prev_day_net_flow,prev_day_senders,prev_day_tx_count,prev_day_volume,pure_whale_tx_count,pure_whale_volume_weth,q25_whale_tx_size_weth,q75_whale_tx_size_weth,quarter,senders_change_pct,senders_ma7,std_whale_tx_size_weth,sustained_inflow,sustained_outflow,total_whale_volume_weth,tx_count_change_pct,tx_count_ma30,tx_count_ma7,tx_count_vs_ma30,tx_count_vs_ma7,ultra_whale_tx_count,ultra_whale_volume_weth,unique_whale_receivers,unique_whale_senders,unusual_activity_flag,volume_change_pct,volume_downtrend,volume_ma30,volume_ma7,volume_range_pct_7d,volume_spike_30d,volume_spike_7d,volume_std30,volume_std7,volume_uptrend,volume_vs_ma30,volume_vs_ma7,volume_zscore_30d,volume_zscore_7d,whale_exchange_deposits_weth,whale_exchange_withdrawals_weth,whale_net_exchange_flow_weth,whale_tx_count,withdrawal_tx_count
count,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.00000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1.096000e+03,1096.0,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1095.000000,1095.000000,1095.000000,1.095000e+03,1096.000000,1.096000e+03,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1.096000e+03,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1.096000e+03,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1.096000e+03,1.096000e+03,1096.000000,1096.000000,1096.000000,1.095000e+03,1.095000e+03,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000,1096.000000
mean,0.081204,87.621060,0.010949,0.073905,17.967425,15.732664,3.998175,25.927007,156.332252,0.864051,0.001094,0.698905,0.090328,0.566606,60.634482,0.164234,1.306052,0.113139,0.09854,0.038321,0.142336,0.915112,34970.680657,47075.708559,43100.890104,0.174784,0.984489,0.011941,341.371350,4.509309e+06,0.0,6.521898,0.000000,-0.244773,-0.081295,335.868293,-210.188134,546.056426,0.070209,1.430767,-0.054197,1879.772603,37482.452968,5.127516e+06,37423.842153,5.138413e+06,0.033864,1.339915,2.508212,0.002803,1882.291341,1000.447464,0.020985,0.748175,5.138719e+06,0.009441,37965.546734,37579.520064,0.993262,0.998904,170.530109,4.005603e+06,1617.047445,1878.729015,0.035584,0.130199,0.275547,4.978316e+06,5.091152e+06,0.966424,0.040146,0.025547,2.396563e+06,1.768995e+06,0.223540,1.035268,1.010970,0.106221,-0.015339,122.304292,122.221144,-0.083146,37457.164234,7.230839
std,0.273273,112.681532,0.104110,0.261736,9.527345,8.805388,1.998629,34.891432,1110.014626,0.342891,0.001437,0.458943,0.286783,0.495770,417.625344,0.522059,7.984723,0.316907,0.29818,0.192058,0.349554,0.058748,19343.855746,31951.274185,32445.618330,1.132079,0.123630,0.011455,248.741706,4.966617e+06,0.0,3.450561,0.479726,33.965855,109.458655,590.827937,638.780939,1037.814262,107.622903,30.000090,378.896939,402.302673,19054.413468,5.042618e+06,19064.574482,5.053888e+06,0.014331,7.991589,1.117697,0.089182,383.897179,675.003695,0.143401,0.434259,5.053941e+06,


Target Variable: next_day_price_direction


KeyError: 'next_day_price_direction'

In [ ]:
### Exploratory Data Analysis (EDA)

# ### 3.1 Target Variable Analysis

# %%
# Plot target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Price direction
df['next_day_price_direction'].value_counts().plot(kind='bar', ax=axes[0])
axes[0].set_title('Next Day Price Direction Distribution')
axes[0].set_xlabel('Direction (0=Down, 1=Up)')
axes[0].set_ylabel('Count')

# Price movement
if 'next_day_price_movement' in df.columns:
    df['next_day_price_movement'].value_counts().plot(kind='bar', ax=axes[1])
    axes[1].set_title('Next Day Price Movement Distribution')
    axes[1].set_xlabel('Movement Category')
    axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()


In [16]:
print(df_whales.columns.tolist())


['accumulation_phase', 'activity_level', 'avg_whale_tx_size_weth', 'bearish_signal', 'block_date', 'bullish_signal', 'cv_whale_tx_size', 'data_quality_score', 'day_of_month', 'day_of_week', 'deposit_tx_count', 'deposit_withdrawal_ratio', 'distribution_phase', 'exchange_tx_ratio', 'extreme_deposit_flag', 'extreme_withdrawal_flag', 'high_concentration_flag', 'inter_exchange_transfers_weth', 'inter_exchange_tx_count', 'iqr_whale_tx_size', 'is_month_end', 'is_month_start', 'is_quarter_end', 'is_weekend', 'known_whale_ratio', 'known_whale_tx_count', 'known_whale_volume_weth', 'max_whale_tx_size_weth', 'median_whale_tx_size_weth', 'mega_whale_dominance_flag', 'mega_whale_ratio', 'mega_whale_tx_count', 'mega_whale_volume_weth', 'min_whale_tx_size_weth', 'month', 'net_flow_direction_change', 'net_flow_ma30', 'net_flow_ma7', 'net_flow_max7', 'net_flow_min7', 'net_flow_range_7d', 'net_flow_vs_ma30', 'net_flow_vs_ma7', 'prev_day_net_flow', 'prev_day_senders', 'prev_day_tx_count', 'prev_day_volume



# %% [markdown]
# ### 3.2 Price and Return Analysis

# %%
# ETH price over time
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

axes[0].plot(df['block_date'], df['eth_price'], label='ETH Price', linewidth=1)
axes[0].set_title('Ethereum Price Over Time')
axes[0].set_ylabel('Price (USD)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Daily returns
axes[1].plot(df['block_date'], df['eth_daily_return'], label='Daily Return', linewidth=0.8, alpha=0.7)
axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[1].set_title('Ethereum Daily Returns')
axes[1].set_ylabel('Return (%)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% [markdown]
# ### 3.3 Whale Transaction Analysis

# %%
# Key whale metrics over time
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Total whale volume
axes[0, 0].plot(df['block_date'], df['total_whale_volume_weth'], linewidth=1)
axes[0, 0].set_title('Total Whale Volume (WETH)')
axes[0, 0].set_ylabel('Volume')
axes[0, 0].grid(True, alpha=0.3)

# Whale transaction count
axes[0, 1].plot(df['block_date'], df['whale_tx_count'], linewidth=1, color='orange')
axes[0, 1].set_title('Whale Transaction Count')
axes[0, 1].set_ylabel('Count')
axes[0, 1].grid(True, alpha=0.3)

# Net exchange flow
axes[1, 0].plot(df['block_date'], df['whale_net_exchange_flow_weth'], linewidth=1, color='green')
axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[1, 0].set_title('Whale Net Exchange Flow (WETH)')
axes[1, 0].set_ylabel('Net Flow')
axes[1, 0].grid(True, alpha=0.3)

# Exchange ratio
axes[1, 1].plot(df['block_date'], df['exchange_tx_ratio'], linewidth=1, color='red')
axes[1, 1].set_title('Exchange Transaction Ratio')
axes[1, 1].set_ylabel('Ratio')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% [markdown]
# ### 3.4 Correlation Analysis

# %%
# Select key features for correlation
key_features = [
    'total_whale_volume_weth', 'whale_tx_count', 'whale_net_exchange_flow_weth',
    'eth_price', 'eth_daily_return', 'eth_volatility_7d', 'eth_momentum_rsi',
    'btc_price', 'btc_daily_return', 'eth_btc_ratio',
    'next_day_return', 'next_day_price_direction'
]

correlation_data = df[key_features].corr()

# Plot correlation heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_data, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix - Key Features')
plt.tight_layout()
plt.show()

# %%
# Correlation with target
target_corr = correlation_data['next_day_price_direction'].sort_values(ascending=False)
print("Top 10 features correlated with next_day_price_direction:")
print(target_corr.head(10))

# %% [markdown]
# ### 3.5 Feature Distribution Analysis

# %%
# Distribution of key features by price direction
features_to_plot = [
    'total_whale_volume_weth', 'whale_net_exchange_flow_weth',
    'eth_momentum_rsi', 'eth_volatility_7d'
]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for i, feature in enumerate(features_to_plot):
    for direction in [0, 1]:
        data = df[df['next_day_price_direction'] == direction][feature]
        axes[i].hist(data, alpha=0.5, bins=30, 
                    label=f"Direction={'Up' if direction else 'Down'}")
    axes[i].set_title(f'{feature} Distribution by Price Direction')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 4. Feature Engineering & Selection

# %%
# Prepare features
print("Preparing features...")

# Make a copy
data = df.copy()

# Encode categorical features
if 'activity_level' in data.columns:
    le = LabelEncoder()
    data['activity_level_encoded'] = le.fit_transform(data['activity_level'])

# Select features (exclude target and date columns)
target_cols = ['next_day_return', 'next_day_price_direction', 
              'next_day_price_movement', 'block_date']

feature_cols = [col for col in data.columns 
               if col not in target_cols and data[col].dtype != 'object']

print(f"Total features: {len(feature_cols)}")

# %%
# Handle missing values
missing_summary = data[feature_cols].isnull().sum().sort_values(ascending=False)
print(f"Features with missing values: {(missing_summary > 0).sum()}")
print("\nTop 10 features with missing values:")
print(missing_summary.head(10))

# Fill missing values
data[feature_cols] = data[feature_cols].fillna(data[feature_cols].median())

# Handle infinite values
data[feature_cols] = data[feature_cols].replace([np.inf, -np.inf], np.nan)
data[feature_cols] = data[feature_cols].fillna(data[feature_cols].median())

print("\nMissing values after imputation:", data[feature_cols].isnull().sum().sum())

# %% [markdown]
# ## 5. Model Selection and Training

# %% [markdown]
# ### 5.1 Prepare Train/Test Split

# %%
# Remove rows with missing target
data_clean = data.dropna(subset=['next_day_price_direction'])

# Prepare X and y
X = data_clean[feature_cols].values
y = data_clean['next_day_price_direction'].values

# Time-series aware split (no shuffling)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Training date range: {data_clean['block_date'].iloc[:split_idx].min()} to {data_clean['block_date'].iloc[:split_idx].max()}")
print(f"Test date range: {data_clean['block_date'].iloc[split_idx:].min()} to {data_clean['block_date'].iloc[split_idx:].max()}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# %% [markdown]
# ### 5.2 Baseline Models

# %%
# Define models to compare
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBClassifier(n_estimators=100, random_state=42),
    'LightGBM': lgb.LGBMClassifier(n_estimators=100, random_state=42, verbose=-1)
}

# Train and evaluate each model
results = []

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train_scaled, y_train)
    
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'AUC-ROC': auc
    })
    
    print(f"Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}, AUC: {auc:.4f}")

# Display results
results_df = pd.DataFrame(results)
print("\n" + "="*70)
print("Model Comparison Results")
print("="*70)
print(results_df.to_string(index=False))

# %% [markdown]
# ### 5.3 Hyperparameter Tuning (Best Model)

# %%
# Select best model (XGBoost) and tune parameters
from sklearn.model_selection import GridSearchCV

print("Tuning XGBoost hyperparameters...")

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_model = xgb.XGBClassifier(random_state=42)

grid_search = GridSearchCV(
    xgb_model, param_grid, cv=5, scoring='roc_auc',
    n_jobs=-1, verbose=1
)

grid_search.fit(X_train_scaled, y_train)

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best cross-validation AUC: {grid_search.best_score_:.4f}")

# Use best model
best_model = grid_search.best_estimator_

# %% [markdown]
# ### 5.4 Final Model Evaluation

# %%
# Evaluate best model
y_pred = best_model.predict(X_test_scaled)
y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]

print("Final Model Performance:")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# %% [markdown]
# ### 5.5 Feature Importance

# %%
# Get feature importance
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

# Plot top 20 features
plt.figure(figsize=(10, 12))
top_features = feature_importance.head(20)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Importance')
plt.title('Top 20 Most Important Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 20 Most Important Features:")
print(feature_importance.head(20).to_string(index=False))

# %% [markdown]
# ## 6. Model Validation

# %%
# Time-series cross-validation
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)
cv_scores = []

for train_idx, val_idx in tscv.split(X_train_scaled):
    X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    model = xgb.XGBClassifier(**grid_search.best_params_, random_state=42)
    model.fit(X_tr, y_tr)
    
    score = roc_auc_score(y_val, model.predict_proba(X_val)[:, 1])
    cv_scores.append(score)

print(f"Time Series CV AUC Scores: {cv_scores}")
print(f"Mean CV AUC: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

# %% [markdown]
# ## 7. Conclusions and Next Steps
# 
# ### Key Findings:
# 1. [Add your findings based on the results]
# 2. [Feature importance insights]
# 3. [Model performance summary]
# 
# ### Recommendations:
# 1. [Business recommendations]
# 2. [Model improvement suggestions]
# 3. [Deployment considerations]
# 
# ### Next Steps:
# 1. Deploy model using Flask API
# 2. Set up monitoring and alerting
# 3. Implement continuous retraining pipeline
# 4. A/B testing with live predictions

print("Analysis complete!")